In [221]:
import pandas as pd
import numpy as np
import math
from scipy.spatial import distance
import json

## SERVICIOS WEB

In [222]:
# GET /rs1a
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
req = json.loads(REQUEST)
args = req['args']

if 'voice' not in args:
    print(json.dumps({'No %Voice Provided': None}))
elif 'music' not in args:
    print(json.dumps({'No %Music Provided': None}))
else:
    #Recogo los argumentos (preferencias del usuario) y los convierto a flotantes
    a = float(args['voice'][0])
    b = float(args['music'][0])
    
    #Creo una tupla
    x = (a,b) 
    
    #leo todos los programas
    dfProgramas = pd.read_csv('D1ProgramsVoiceMusicTodos.csv')
    
    #Creo una copia del dataset y añado la columna 'Distancia'
    dfProgramasParecidos = dfProgramas.copy()
    dfProgramasParecidos['Distancia']=-2
    
    #Calculo la distancia entre las preferencias del usuario y las características de cada programa
    for index, row in dfProgramasParecidos.iterrows():
        tuples = [tuple(x) for x in dfProgramasParecidos.iloc[index:index+1, 1:dfProgramasParecidos.shape[1]-1].to_numpy()]
        dfProgramasParecidos.iloc[index, dfProgramasParecidos.columns.get_loc('Distancia')] = distance.euclidean(x,tuples)
    
    #Elimino las columnas del dataset que no necesito
    dfProgramasParecidos.drop(['PorcentajeVoz', 'PorcentajeMusica'], inplace=True, axis=1)
    
    #Escalo las distancias entre 0 y 1 (0 más parecido y 1 menos parecido)
    scaler = MinMaxScaler()
    dfProgramasParecidos[['Distancia']] = scaler.fit_transform(dfProgramasParecidos[['Distancia']])
    
    #Cambio para que la distancia más cercana a 1 sea la más parecido y la más cercana a 0 la más diferente (el recomendador content-based da los resultados en este formato)
    dfProgramasParecidos['Distancia'] = 1 - dfProgramasParecidos['Distancia'] 
    
    #Devuelvo los resultados en json
    jsonImprimir=dfProgramasParecidos.to_json(orient="records")
    print(jsonImprimir)

NameError: name 'REQUEST' is not defined

In [223]:
# GET /rs1b
import json
req = json.loads(REQUEST)
args = req['args']

if 'like' not in args:
    print(json.dumps({'No likes Provided': None}))
else:
    #Recibo a través del servicio web los programas que le gustan al usuario
    programaslikeenbruto = args["like"][0]
    programasLike = programaslikeenbruto.split("|")
    
    #programasLike = args['like'][0]
    #print(programasLike)
    #print(programasLike)
    #leo todos los programas
    dfProgramasVoiceMusicTodos = pd.read_csv('D1ProgramsVoiceMusicTodos.csv')
    
    #Creo un dataframe con las referencias del usuario, recogidas por el servicio web
    #data = {'_nombre':  [programasLike]}
    data = {'_nombre':  programasLike}
    dfUsuario = pd.DataFrame (data, columns = ['_nombre'])
    
    #Obtengo las características de los programas de radio que le gustan al usuario
    dfMatrizGustos = dfProgramasVoiceMusicTodos[dfProgramasVoiceMusicTodos["_nombre"].isin(dfUsuario['_nombre'].to_list())]

    #Creo el vector de preferencias del usuario y lo normalizo entre 0 y 1
    vectorPreferencias = dfMatrizGustos.sum(axis=0)
    vectorPreferencias = vectorPreferencias.drop(['_nombre']);
    vectorPreferencias = vectorPreferencias / vectorPreferencias.max()
    
    #Quito la columna de nombre en los gustos del usuario para poder multiplicar con el vector de preferencias
    dfMatrizGustosSinNombre = dfProgramasVoiceMusicTodos.drop(['_nombre'], axis=1)

    #Multiplico todos los programas con sus características con el vector de preferencias del usuario
    dfRecomendacionesVoiceMusic = dfMatrizGustosSinNombre.dot(vectorPreferencias)
    dfRecomendacionesVoiceMusic = dfRecomendacionesVoiceMusic.to_frame(name="gradoRecomendacion")
    
    # ?
    dfRecomendacionesVoiceMusic['_nombre'] = dfProgramasVoiceMusicTodos['_nombre']
    
    #Escalo las distancias entre 0 y 1 
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    dfRecomendacionesVoiceMusic[['gradoRecomendacion']] = scaler.fit_transform(dfRecomendacionesVoiceMusic[['gradoRecomendacion']])
    dfRecomendacionesVoiceMusic
    
    #Devuelvo las recomendaciones en JSON
    jsonImprimir=dfRecomendacionesVoiceMusic.to_json(orient="records")
    print(jsonImprimir)

NameError: name 'REQUEST' is not defined

In [224]:
# GET /rs2a
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import logging
from http.server import BaseHTTPRequestHandler, HTTPServer
from urllib.parse import urlparse

#leo todos los programas
dfProgramasTopicsTodos = pd.read_csv('D2ProgramsGenresTodos.csv')

#Creo una copia del dataset y añado la columna 'Distancia'
dfProgramasParecidos = dfProgramasTopicsTodos.copy()
dfProgramasParecidos['Distancia']=-2

#Obtengo todos los géneros existentes         
todos_generos = pd.DataFrame(columns=dfProgramasTopicsTodos.columns)
todos_generos.loc[len(todos_generos)] = 0
todos_generos.drop(['_nombre'], axis=1, inplace=True)
todos_generos

#Recogo los argumentos de la petición HTTP
req = json.loads(REQUEST)
args = req['args']

#Los transformo a su nombre=valor
for arg in args:
    name=arg
    value=args[arg][0]
    todos_generos[name]=float(value)
    #print(f"name: {name}, value: {value}")

#Calculo la distancia entre las preferencias del usuario y las características de cada programa
for index, row in dfProgramasParecidos.iterrows():
    tuples = [tuple(x) for x in dfProgramasParecidos.iloc[index:index+1, 1:dfProgramasParecidos.shape[1]-1].to_numpy()]
    dfProgramasParecidos.iloc[index, dfProgramasParecidos.columns.get_loc('Distancia')] = distance.euclidean(todos_generos,tuples)
    
#Elimino las columnas del dataset que no necesito
#dfProgramasParecidos.drop(['informativo', 'deportes', 'entretenimiento', 'musical', 'educacion'], inplace=True, axis=1)
dfProgramasParecidos = dfProgramasParecidos[["_nombre", "Distancia"]]

#Escalo las distancias entre 0 y 1 (0 más parecido y 1 menos parecido)
scaler = MinMaxScaler()
dfProgramasParecidos[['Distancia']] = scaler.fit_transform(dfProgramasParecidos[['Distancia']])
    
#Cambio para que la distancia más cercana a 1 sea la más parecido y la más cercana a 0 la más diferente (el recomendador content-based da los resultados en este formato)
dfProgramasParecidos['Distancia'] = 1 - dfProgramasParecidos['Distancia'] 
    
#Devuelvo los resultados en json
jsonImprimir=dfProgramasParecidos.to_json(orient="records")
print(jsonImprimir)





NameError: name 'REQUEST' is not defined

In [225]:
# GET /rs2b
import json
req = json.loads(REQUEST)
args = req['args']

if 'like' not in args:
    print(json.dumps({'No likes Provided': None}))
else:
    #Recibo a través del servicio web los programas que le gustan al usuario
    programaslikeenbruto = args["like"][0]
    programasLike = programaslikeenbruto.split("|")
    
    #programasLike = args['like'][0]
    #print(programasLike)
    #print(programasLike)
    #leo todos los programas
    dfProgramasGenerosTodos = pd.read_csv('D2ProgramsGenresTodos.csv')
    
    #Creo un dataframe con las referencias del usuario, recogidas por el servicio web
    #data = {'_nombre':  [programasLike]}
    data = {'_nombre':  programasLike}
    dfUsuario = pd.DataFrame (data, columns = ['_nombre'])
    
    #Obtengo las características de los programas de radio que le gustan al usuario
    dfMatrizGustos = dfProgramasGenerosTodos[dfProgramasGenerosTodos["_nombre"].isin(dfUsuario['_nombre'].to_list())]

    #Creo el vector de preferencias del usuario y lo normalizo entre 0 y 1
    vectorPreferencias = dfMatrizGustos.sum(axis=0)
    vectorPreferencias = vectorPreferencias.drop(['_nombre']);
    vectorPreferencias = vectorPreferencias / vectorPreferencias.max()
    
    #Quito la columna de nombre en los gustos del usuario para poder multiplicar con el vector de preferencias
    dfMatrizGustosSinNombre = dfProgramasGenerosTodos.drop(['_nombre'], axis=1)

    #Multiplico todos los programas con sus características con el vector de preferencias del usuario
    dfRecomendaciones = dfMatrizGustosSinNombre.dot(vectorPreferencias)
    dfRecomendaciones = dfRecomendaciones.to_frame(name="gradoRecomendacion")
    
    # ?
    dfRecomendaciones['_nombre'] = dfProgramasGenerosTodos['_nombre']
    
    #Escalo las distancias entre 0 y 1 
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    dfRecomendaciones[['gradoRecomendacion']] = scaler.fit_transform(dfRecomendaciones[['gradoRecomendacion']])
    dfRecomendaciones
    
    #Devuelvo las recomendaciones en JSON
    jsonImprimir=dfRecomendaciones.to_json(orient="records")
    print(jsonImprimir)

NameError: name 'REQUEST' is not defined

In [226]:
# GET /rs3a
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
req = json.loads(REQUEST)
args = req['args']

if 'news' not in args:
    print(json.dumps({'No %news Provided': None}))
elif 'sport' not in args:
    print(json.dumps({'No %sport Provided': None}))
elif 'entertainment' not in args:
    print(json.dumps({'No %entertainment Provided': None}))
elif 'musical' not in args:
    print(json.dumps({'No %musical Provided': None}))
elif 'education' not in args:
    print(json.dumps({'No %education Provided': None}))    
else:
    #Recogo los argumentos (preferencias del usuario) y los convierto a flotantes
    a = float(args['news'][0])
    b = float(args['sport'][0])
    c = float(args['entertainment'][0])
    d = float(args['musical'][0])
    e = float(args['education'][0])    
    
    #Creo una tupla
    x = (a, b, c, d, e)
    
    #leo todos los programas
    dfProgramasTopicsTodos = pd.read_csv('D3ProgramsTopicsTodos.csv')
    
    #Creo una copia del dataset y añado la columna 'Distancia'
    dfProgramasParecidos = dfProgramasTopicsTodos.copy()
    dfProgramasParecidos['Distancia']=-2
    
    #Calculo la distancia entre las preferencias del usuario y las características de cada programa
    for index, row in dfProgramasParecidos.iterrows():
        tuples = [tuple(x) for x in dfProgramasParecidos.iloc[index:index+1, 1:dfProgramasParecidos.shape[1]-1].to_numpy()]
        dfProgramasParecidos.iloc[index, dfProgramasParecidos.columns.get_loc('Distancia')] = distance.euclidean(x,tuples)
    
    #Elimino las columnas del dataset que no necesito
    dfProgramasParecidos.drop(['informativo', 'deportes', 'entretenimiento', 'musical', 'educacion'], inplace=True, axis=1)
    
    #Escalo las distancias entre 0 y 1 (0 más parecido y 1 menos parecido)
    scaler = MinMaxScaler()
    dfProgramasParecidos[['Distancia']] = scaler.fit_transform(dfProgramasParecidos[['Distancia']])
    
    #Cambio para que la distancia más cercana a 1 sea la más parecido y la más cercana a 0 la más diferente (el recomendador content-based da los resultados en este formato)
    dfProgramasParecidos['Distancia'] = 1 - dfProgramasParecidos['Distancia'] 
    
    #Devuelvo los resultados en json
    jsonImprimir=dfProgramasParecidos.to_json(orient="records")
    print(jsonImprimir)

NameError: name 'REQUEST' is not defined

In [227]:
# GET /rs3b
import json
req = json.loads(REQUEST)
args = req['args']

if 'like' not in args:
    print(json.dumps({'No likes Provided': None}))
else:
    #Recibo a través del servicio web los programas que le gustan al usuario
    programaslikeenbruto = args["like"][0]
    programasLike = programaslikeenbruto.split("|")
    
    #programasLike = args['like'][0]
    #print(programasLike)
    #print(programasLike)
    #leo todos los programas
    dfProgramasTopicsTodos = pd.read_csv('D3ProgramsTopicsTodos.csv')
    
    #Creo un dataframe con las referencias del usuario, recogidas por el servicio web
    #data = {'_nombre':  [programasLike]}
    data = {'_nombre':  programasLike}
    dfUsuario = pd.DataFrame (data, columns = ['_nombre'])
    
    #Obtengo las características de los programas de radio que le gustan al usuario
    dfMatrizGustos = dfProgramasTopicsTodos[dfProgramasTopicsTodos["_nombre"].isin(dfUsuario['_nombre'].to_list())]

    #Creo el vector de preferencias del usuario y lo normalizo entre 0 y 1
    vectorPreferencias = dfMatrizGustos.sum(axis=0)
    vectorPreferencias = vectorPreferencias.drop(['_nombre']);
    vectorPreferencias = vectorPreferencias / vectorPreferencias.max()
    
    #Quito la columna de nombre en los gustos del usuario para poder multiplicar con el vector de preferencias
    dfMatrizGustosSinNombre = dfProgramasTopicsTodos.drop(['_nombre'], axis=1)

    #Multiplico todos los programas con sus características con el vector de preferencias del usuario
    dfRecomendacionesTopics = dfMatrizGustosSinNombre.dot(vectorPreferencias)
    dfRecomendacionesTopics = dfRecomendacionesTopics.to_frame(name="gradoRecomendacion")
    
    # ?
    dfRecomendacionesTopics['_nombre'] = dfProgramasTopicsTodos['_nombre']
    
    #Escalo las distancias entre 0 y 1 
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    dfRecomendacionesTopics[['gradoRecomendacion']] = scaler.fit_transform(dfRecomendacionesTopics[['gradoRecomendacion']])
    dfRecomendacionesTopics
    
    #Devuelvo las recomendaciones en JSON
    jsonImprimir=dfRecomendacionesTopics.to_json(orient="records")
    print(jsonImprimir)

NameError: name 'REQUEST' is not defined

In [228]:
# GET /rs4a
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
req = json.loads(REQUEST)
args = req['args']

if 'anger' not in args:
    print(json.dumps({'No %anger Provided': None}))
elif 'fear' not in args:
    print(json.dumps({'No %fear Provided': None}))
elif 'joy' not in args:
    print(json.dumps({'No %joy Provided': None}))
elif 'sadness' not in args:
    print(json.dumps({'No %sadness Provided': None}))
elif 'analytical' not in args:
    print(json.dumps({'No %analytical Provided': None}))
elif 'confident' not in args:
    print(json.dumps({'No %confident Provided': None}))
elif 'tentative' not in args:
    print(json.dumps({'No %tentative Provided': None}))    
else:
    #Recogo los argumentos (preferencias del usuario) y los convierto a flotantes
    a = float(args['anger'][0])
    b = float(args['fear'][0])
    c = float(args['joy'][0])
    d = float(args['sadness'][0])
    e = float(args['analytical'][0])    
    f = float(args['confident'][0])  
    g = float(args['tentative'][0])  
    
    #Creo una tupla
    x = (a, b, c, d, e, f, g)
    
    #leo todos los programas
    dfProgramasTonesTodos = pd.read_csv('D4ProgramsTonesTodos.csv')
    
    #Creo una copia del dataset y añado la columna 'Distancia'
    dfProgramasParecidos = dfProgramasTonesTodos.copy()
    dfProgramasParecidos['Distancia']=-2
    
    #Calculo la distancia entre las preferencias del usuario y las características de cada programa
    for index, row in dfProgramasParecidos.iterrows():
        tuples = [tuple(x) for x in dfProgramasParecidos.iloc[index:index+1, 1:dfProgramasParecidos.shape[1]-1].to_numpy()]
        dfProgramasParecidos.iloc[index, dfProgramasParecidos.columns.get_loc('Distancia')] = distance.euclidean(x,tuples)
    
    #Elimino las columnas del dataset que no necesito
    dfProgramasParecidos.drop(['anger', 'fear', 'joy', 'sadness', 'analytical', 'confident', 'tentative'], inplace=True, axis=1)
    
    #Escalo las distancias entre 0 y 1 (0 más parecido y 1 menos parecido)
    scaler = MinMaxScaler()
    dfProgramasParecidos[['Distancia']] = scaler.fit_transform(dfProgramasParecidos[['Distancia']])
    
    #Cambio para que la distancia más cercana a 1 sea la más parecido y la más cercana a 0 la más diferente (el recomendador content-based da los resultados en este formato)
    dfProgramasParecidos['Distancia'] = 1 - dfProgramasParecidos['Distancia'] 
    
    #Devuelvo los resultados en json
    jsonImprimir=dfProgramasParecidos.to_json(orient="records")
    print(jsonImprimir)

NameError: name 'REQUEST' is not defined

In [229]:
# GET /rs4b
import json
req = json.loads(REQUEST)
args = req['args']

if 'like' not in args:
    print(json.dumps({'No likes Provided': None}))
else:
    #Recibo a través del servicio web los programas que le gustan al usuario
    programaslikeenbruto = args["like"][0]
    programasLike = programaslikeenbruto.split("|")
    
    #programasLike = args['like'][0]
    #print(programasLike)
    #print(programasLike)
    #leo todos los programas
    dfProgramasTonesTodos = pd.read_csv('D4ProgramsTonesTodos.csv')
    
    #Creo un dataframe con las referencias del usuario, recogidas por el servicio web
    #data = {'_nombre':  [programasLike]}
    data = {'_nombre':  programasLike}
    dfUsuario = pd.DataFrame (data, columns = ['_nombre'])
    
    #Obtengo las características de los programas de radio que le gustan al usuario
    dfMatrizGustos = dfProgramasTonesTodos[dfProgramasTonesTodos["_nombre"].isin(dfUsuario['_nombre'].to_list())]

    #Creo el vector de preferencias del usuario y lo normalizo entre 0 y 1
    vectorPreferencias = dfMatrizGustos.sum(axis=0)
    vectorPreferencias = vectorPreferencias.drop(['_nombre']);
    vectorPreferencias = vectorPreferencias / vectorPreferencias.max()
    
    #Quito la columna de nombre en los gustos del usuario para poder multiplicar con el vector de preferencias
    dfMatrizGustosSinNombre = dfProgramasTonesTodos.drop(['_nombre'], axis=1)

    #Multiplico todos los programas con sus características con el vector de preferencias del usuario
    dfRecomendacionesTones = dfMatrizGustosSinNombre.dot(vectorPreferencias)
    dfRecomendacionesTones = dfRecomendacionesTones.to_frame(name="gradoRecomendacion")
    
    # ?
    dfRecomendacionesTones['_nombre'] = dfProgramasTonesTodos['_nombre']
    
    #Escalo las distancias entre 0 y 1 
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    dfRecomendacionesTones[['gradoRecomendacion']] = scaler.fit_transform(dfRecomendacionesTones[['gradoRecomendacion']])
    dfRecomendacionesTones
    
    #Devuelvo las recomendaciones en JSON
    jsonImprimir=dfRecomendacionesTones.to_json(orient="records")
    print(jsonImprimir)

NameError: name 'REQUEST' is not defined

In [ ]:
# GET /music_genres
genres_dataframe = pd.read_csv("D2ProgramsGenresTodos.csv")
genres = list(genres_dataframe.columns)[1:]
print(json.dumps(genres))